# MODELO DE RECOMENDACION DE PELICULAS

Primero vamos a trabajar el modelo de Machine Learning en un Notebook, para posteriormente incluirlo en el script mail.py 

In [242]:
# Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity



In [243]:
# Cargar la Informacion
# Función que carga los datos desde un archivo Parquet
def cargar_datos():
    # Ajusta la ruta al archivo Parquet
    df = pd.read_parquet('./Datasets/data_movies.parquet')
    df_credits = pd.read_parquet('./Datasets/data_credits.parquet')
    return df, df_credits

#Cargo los Datos
peliculas, creditos = cargar_datos()

In [244]:
#Hago un merge de los dos datasets para facilitar el trabajo en ellos

data = pd.merge(peliculas, creditos, on ='id')
data_filtrada = data[data['release_year'] > 2000] #Filtro el dataset cpor año de lanzamiento mayor al 2000
data_filtrada.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'spoken_languages_1_iso',
       'spoken_languaje_1_name', 'production_companies_1_name',
       'production_companies_1_id', 'production_companies_2_name',
       'production_companies_2_id', 'production_countries_1_iso',
       'production_countries_1_name', 'genres_1_id', 'genres_1_name',
       'genres_2_id', 'genres_2_name', 'genres_3_id', 'genres_3_name',
       'release_year', 'return', 'retorno_US', 'cast_name', 'cast_name_2',
       'cast_name_3', 'cast_name_4', 'cast_name_5', 'cast_name_6',
       'cast_name_7', 'cast_name_8', 'cast_name_9', 'cast_name_10',
       'director'],
      dtype='object')

In [245]:
columnas_eliminar = ('budget','release_date',
                                    'revenue',
                                    'runtime', 
                                    'status',
                                    'tagline',
                                    'vote_count', 
                                    'belongs_to_collection_id',
                                    'belongs_to_collection_name', 
                                    'spoken_languages_1_iso',
                                    'spoken_languaje_1_name', 
                                    'production_companies_1_name',
                                    'production_companies_1_id', 
                                    'production_companies_2_name',
                                    'production_companies_2_id', 
                                    'production_countries_1_iso',
                                    'production_countries_1_name', 
                                    'genres_1_id',
                                    'genres_2_id', 
                                    'genres_2_name', 
                                    'genres_3_id', 
                                    'genres_3_name',
                                    'return', 
                                    'retorno_US',
                                    'cast_name_2',
                                    'cast_name_3', 
                                    'cast_name_4', 
                                    'cast_name_5', 
                                    'cast_name_6',
                                    'cast_name_7', 
                                    'cast_name_8', 
                                    'cast_name_9', 
                                    'cast_name_10')
for i in columnas_eliminar:
    data_filtrada = data_filtrada.drop(i, axis=1)

data_filtrada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23214 entries, 707 to 45374
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 23214 non-null  int64  
 1   original_language  23208 non-null  object 
 2   overview           22696 non-null  object 
 3   popularity         23214 non-null  float64
 4   title              23214 non-null  object 
 5   vote_average       23214 non-null  float64
 6   genres_1_name      22156 non-null  object 
 7   release_year       23214 non-null  int32  
 8   cast_name          23214 non-null  object 
 9   director           23214 non-null  object 
dtypes: float64(2), int32(1), int64(1), object(6)
memory usage: 1.9+ MB


In [246]:
# Tomo las características relevantes Features:

features = ['title',
            'overview',
            'director',
            'cast_name',
            'original_language',
            'genres_1_name',
            'popularity',
            'vote_average',
            'genres_1_name', 
            'cast_name',
            'director',
            'release_year'
            ]

# Llenado de valores nulos
for feature in features:
    if data_filtrada[feature].dtype == 'object':
        data_filtrada[feature] = data[feature].fillna('')

In [247]:
# Preproceso la información: 
# Llenado de valores nulos
for feature in features:
    if data_filtrada[feature].dtype == 'object':
        data_filtrada[feature] = data[feature].fillna('')

In [248]:
# Creo una columna con los strigs combinados (texto_total): Tomo solo algunos

data_filtrada['texto_combinado'] = data_filtrada['overview'] + ' ' + data_filtrada['genres_1_name']  + ' ' + data_filtrada['cast_name'] + ' ' + data_filtrada['director']

#  vectorización de características textuales
tfidf = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf.fit_transform(data_filtrada['texto_combinado'])


In [249]:
# Estandarización de los datos numericos:

scaler = StandardScaler()  #Instancio 
data_filtrada[['popularity','release_year']] = scaler.fit_transform(data_filtrada[['popularity', 'release_year']])

In [250]:
#Creo una matriz total uniendo las catacteristicas categóricas más las numéricas.
matrix_total = np.hstack((tfidf_matrix.toarray(), data_filtrada[['popularity', 'release_year']].values))

MemoryError: Unable to allocate 11.8 GiB for an array with shape (23214, 68331) and data type float64

In [ ]:
def recomendacion(titulo):

    idx = data_filtrada.index[data_filtrada['title']==titulo].to_list()[0]

    #Calcula la matrix de similitud
    similarity_matrix = cosine_similarity(matrix_total, matrix_total)

    #obtengo los paresde peliculas indice y score
    similarity_score = list(enumerate(similarity_matrix[idx]))

    #Ordeno las peliculas por puntaje de similitud
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    #Tomo los indices de las 5 peliculas similares. quito la primera [0] que es la misma dada
    similar_movies_indices= [i[0] for i in similarity_score[1:6]]

    return data_filtrada['title'].iloc[similar_movies_indices].tolist()



In [ ]:
titulo_pelicula = 'two friends'

recomendaciones = recomendacion(titulo_pelicula)
print("Películas recomendadas para '{}':".format(titulo_pelicula))
for pelicula in recomendaciones:
    print(pelicula)

Películas recomendadas para 'two friends':
harry potter and the philosopher's stone
spirited away
the lord of the rings: the fellowship of the ring
harry potter and the chamber of secrets
the lord of the rings: the two towers


In [ ]:
data_filtrada['title']

707               two friends
867                    venice
1080            the sleepover
2112        the farmer's wife
2562     a place at the table
                 ...         
45363                     mom
45364       the morning after
45371     century of birthing
45372                betrayal
45374                queerama
Name: title, Length: 23214, dtype: object

In [ ]:
data_filtrada.columns


Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'spoken_languages_1_iso',
       'spoken_languaje_1_name', 'production_companies_1_name',
       'production_companies_1_id', 'production_companies_2_name',
       'production_companies_2_id', 'production_countries_1_iso',
       'production_countries_1_name', 'genres_1_id', 'genres_1_name',
       'genres_2_id', 'genres_2_name', 'genres_3_id', 'genres_3_name',
       'release_year', 'return', 'retorno_US', 'cast_name', 'cast_name_2',
       'cast_name_3', 'cast_name_4', 'cast_name_5', 'cast_name_6',
       'cast_name_7', 'cast_name_8', 'cast_name_9', 'cast_name_10', 'director',
       'texto_combinado'],
      dtype='object')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45375 entries, 0 to 45374
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   budget                       45375 non-null  int64  
 1   id                           45375 non-null  int64  
 2   original_language            45375 non-null  object 
 3   overview                     45375 non-null  object 
 4   popularity                   45375 non-null  float64
 5   release_date                 45375 non-null  object 
 6   revenue                      45375 non-null  float64
 7   runtime                      45129 non-null  float64
 8   status                       45295 non-null  object 
 9   tagline                      20398 non-null  object 
 10  title                        45375 non-null  object 
 11  vote_average                 45375 non-null  float64
 12  vote_count                   45375 non-null  float64
 13  belongs_to_colle

In [ ]:
data.head()

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,cast_name_3,cast_name_4,cast_name_5,cast_name_6,cast_name_7,cast_name_8,cast_name_9,cast_name_10,director,texto_combinado
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,None,...,don rickles,jim varney,wallace shawn,john ratzenberger,annie potts,john morris,erik von detten,laurie metcalf,john lasseter,"Led by Woody, Andy's toys live happily in his ..."
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,kirsten dunst,bradley pierce,bonnie hunt,bebe neuwirth,david alan grier,patricia clarkson,adam hann-byrd,laura bell bundy,larry j. franco,When siblings Judy and Peter discover an encha...
2,0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,ann-margret,sophia loren,daryl hannah,burgess meredith,kevin pollak,none,none,none,howard deutch,A family wedding reignites the ancient feud be...
3,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,loretta devine,lela rochon,gregory hines,dennis haysbert,michael beach,mykelti williamson,lamont johnson,wesley snipes,forest whitaker,"Cheated on, mistreated and stepped on, the wom..."
4,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,martin short,kimberly williams-paisley,george newbern,kieran culkin,bd wong,peter michael goetz,kate mcgregor-stewart,jane adams,alan silvestri,Just when George Banks has recovered from his ...
